In [17]:
# A glimpse of current classification model

from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import torch

model_id = "google/vit-base-patch16-224"

CKPT = "./vit-single-out/best"   # change if yours is elsewhere
IMG  = "./classification_dataset/images/2015_26395.jpg"  # the image you want to test

device = "cuda" if torch.cuda.is_available() else "cpu"
processor = AutoImageProcessor.from_pretrained(CKPT)
model = AutoModelForImageClassification.from_pretrained(CKPT).to(device).eval()

img = Image.open(IMG).convert("RGB")
inputs = processor(images=img, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
pred_id = int(logits.argmax(-1))
print(model.config.id2label[pred_id]) 


crab


In [14]:
import torch, os

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # nicer OOM behavior


CUDA available: False


In [ ]:
# Pipelining data and image 
with open("./classification_dataset/labels.txt") as f:
    lines = f.readlines()

def sort_key(line):
    parts = line.split()
    if not parts:
        return (9999, 999999)

    prefix, rest = line.split('_')
    number = int(rest.split('.')[0])
    return (int(prefix), number)

lines.sort(key=sort_key) 
print(lines[:20000])
data = [line[0:-1] for line in lines]
print(data[:20]) # Clean data

# Create correct pathname
features = [line.split(' ')[0] for line in data]
labels = [line.split(' ')[1] for line in data]
print(features[:20])
print(labels[:20])

['2015_9.jpg roundfish\n', '2015_18.jpg whelk\n', '2015_32.jpg roundfish\n', '2015_40.jpg skate\n', '2015_45.jpg roundfish\n', '2015_47.jpg skate\n', '2015_102.jpg Scallop\n', '2015_117.jpg flatfish\n', '2015_126.jpg roundfish\n', '2015_128.jpg Scallop\n', '2015_136.jpg roundfish\n', '2015_142.jpg roundfish\n', '2015_151.jpg flatfish\n', '2015_155.jpg whelk\n', '2015_156.jpg whelk\n', '2015_157.jpg Scallop\n', '2015_161.jpg Scallop\n', '2015_170.jpg Scallop\n', '2015_228.jpg whelk\n', '2015_232.jpg whelk\n', '2015_258.jpg Scallop\n', '2015_297.jpg whelk\n', '2015_310.jpg Scallop\n', '2015_312.jpg roundfish\n', '2015_453.jpg skate\n', '2015_461.jpg skate\n', '2015_464.jpg roundfish\n', '2015_467.jpg roundfish\n', '2015_480.jpg Scallop\n', '2015_482.jpg flatfish\n', '2015_491.jpg Scallop\n', '2015_496.jpg whelk\n', '2015_510.jpg roundfish\n', '2015_526.jpg flatfish\n', '2015_528.jpg flatfish\n', '2015_529.jpg roundfish\n', '2015_530.jpg roundfish\n', '2015_533.jpg roundfish\n', '2015_537

In [ ]:
try:
    from sklearn.model_selection import train_test_split
    HAVE_SKLEARN = True
except Exception:
    HAVE_SKLEARN = False

print(HAVE_SKLEARN)

In [ ]:
import os, csv, random
from dataclasses import dataclass
from typing import Dict, Any, List, Tuple

import torch
from torch.utils.data import Dataset
from PIL import Image

try:
    from sklearn.model_selection import train_test_split
    HAVE_SKLEARN = True
except Exception:
    HAVE_SKLEARN = False

from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    TrainingArguments,
    Trainer,
    DefaultDataCollator,
)

# Hyperparameters
model_id = "google/vit-base-patch16-224"
seed = 42
batch_size = 16
num_epochs = 5
learning_rate = 5e-5
weight_decay = 0.05
warmup_ratio = 0.1

# Create correct directory to the image file
image_root = "./classification_dataset/images/"
pairs = [(image_root + features[i], labels[i]) for i in range(len(features))]
print(pairs[:20])
image_dir = os.path.join(image_root, "images")
labels_path = os.path.join(image_root, "labels.txt")

# Classes
classes = sorted(list({label for _, label in pairs}))
label2id = {c: i for i, c in enumerate(classes)}
id2label = {i: c for c, i in label2id.items()}
# print(label2id)
# print(id2label)

# Optional: stratified split (recommended)
if HAVE_SKLEARN:
    X = [p for p, _ in pairs]
    y = [label2id[lbl] for _, lbl in pairs]
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        X, y, test_size=0.1, random_state=seed, stratify=y
    )
    train_pairs = list(zip(train_paths, [id2label[i] for i in train_labels]))
    val_pairs   = list(zip(val_paths,   [id2label[i] for i in val_labels]))
else:
    # fallback: simple shuffle + 90/10 split
    random.shuffle(pairs)
    k = max(1, int(0.1 * len(pairs)))
    val_pairs = pairs[:k]
    train_pairs = pairs[k:]

print(f"Train: {len(train_pairs)} | Val: {len(val_pairs)} | Classes: {classes}")

# -----------------------------
# Processor & Dataset
# -----------------------------
processor = AutoImageProcessor.from_pretrained(model_id)

@dataclass
class SingleFolderDataset(Dataset):
    pairs: List[Tuple[str, str]]
    processor: AutoImageProcessor
    label2id: Dict[str, int]

    def __len__(self) -> int:
        return len(self.pairs)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        img_path, label = self.pairs[idx]
        with Image.open(img_path) as im:
            img = im.convert("RGB")
        enc = self.processor(images=img, return_tensors="pt")
        return {
            "pixel_values": enc["pixel_values"].squeeze(0),
            "labels": torch.tensor(self.label2id[label], dtype=torch.long),
        }

train_ds = SingleFolderDataset(train_pairs, processor, label2id)
val_ds   = SingleFolderDataset(val_pairs,   processor, label2id)

# -----------------------------
# Model
# -----------------------------
model = AutoModelForImageClassification.from_pretrained(
    model_id,
    num_labels=len(classes),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

# -----------------------------
# Metrics
# -----------------------------
def accuracy(eval_pred):
    logits, labels = eval_pred
    import numpy as np
    preds = logits.argmax(-1)
    return {"accuracy": (preds == labels).mean().item()}

# -----------------------------
# Training
# -----------------------------
args = TrainingArguments(
    output_dir="./vit-single-out",
    remove_unused_columns=False,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    warmup_ratio=warmup_ratio,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=20,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    dataloader_pin_memory=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=DefaultDataCollator(),
    compute_metrics=accuracy,
)

trainer.train()
metrics = trainer.evaluate()
print("Final eval metrics:", metrics)

# Save best checkpoint
trainer.save_model("./vit-single-out/best")
processor.save_pretrained("./vit`-single-out/best")


[('./classification_dataset/images/2015_9.jpg', 'roundfish'), ('./classification_dataset/images/2015_18.jpg', 'whelk'), ('./classification_dataset/images/2015_32.jpg', 'roundfish'), ('./classification_dataset/images/2015_40.jpg', 'skate'), ('./classification_dataset/images/2015_45.jpg', 'roundfish'), ('./classification_dataset/images/2015_47.jpg', 'skate'), ('./classification_dataset/images/2015_102.jpg', 'Scallop'), ('./classification_dataset/images/2015_117.jpg', 'flatfish'), ('./classification_dataset/images/2015_126.jpg', 'roundfish'), ('./classification_dataset/images/2015_128.jpg', 'Scallop'), ('./classification_dataset/images/2015_136.jpg', 'roundfish'), ('./classification_dataset/images/2015_142.jpg', 'roundfish'), ('./classification_dataset/images/2015_151.jpg', 'flatfish'), ('./classification_dataset/images/2015_155.jpg', 'whelk'), ('./classification_dataset/images/2015_156.jpg', 'whelk'), ('./classification_dataset/images/2015_157.jpg', 'Scallop'), ('./classification_dataset

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([7]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\Conda\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [28]:
metrics = trainer.evaluate()
print(metrics)

d:\Conda\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyboardInterrupt: 

In [ ]:
# Scoring Accuracy
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import torch
from tqdm import tqdm

with open("./classification_dataset/labels.txt") as f:
    lines = f.readlines()

def sort_key(line):
    parts = line.split()
    if not parts:
        return (9999, 999999)

    prefix, rest = line.split('_')
    number = int(rest.split('.')[0])
    return (int(prefix), number)

lines.sort(key=sort_key) 
data = [line[0:-1] for line in lines]

# Create correct pathname
features = [line.split(' ')[0] for line in data]
labels = [line.split(' ')[1] for line in data]
indices = random.sample([i for i in range(len(features))], 100)

model_id = "google/vit-base-patch16-224"
CKPT = "./vit-single-out/best"   # change if yours is elsewhere
correct_infer = 0 

for i in tqdm(indices):
    IMG  = f"./classification_dataset/images/{features[i]}"  # the image you want to test

    device = "cuda" if torch.cuda.is_available() else "cpu"
    processor = AutoImageProcessor.from_pretrained(CKPT)
    model = AutoModelForImageClassification.from_pretrained(CKPT).to(device).eval()

    img = Image.open(IMG).convert("RGB")
    inputs = processor(images=img, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    pred_id = int(logits.argmax(-1))
    if model.config.id2label[pred_id] == labels[i]:
        correct_infer += 1
    else:
        print(model.config.id2label[pred_id], labels[i])

accuracy = correct_infer / 1000 * 100
print(f"The total accuracy is {accuracy}")


 85%|████████▌ | 85/100 [00:34<00:05,  2.86it/s]

roundfish crab


100%|██████████| 100/100 [00:39<00:00,  2.53it/s]

The total accuracy is 0.9428571428571428


In [30]:
# Scoring on validation set
import os, csv, random
from dataclasses import dataclass
from typing import Dict, Any, List, Tuple

from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import torch
from tqdm import tqdm

import torch
from torch.utils.data import Dataset
from PIL import Image

try:
    from sklearn.model_selection import train_test_split
    HAVE_SKLEARN = True
except Exception:
    HAVE_SKLEARN = False

from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    TrainingArguments,
    Trainer,
    DefaultDataCollator,
)

# Hyperparameters
model_id = "google/vit-base-patch16-224"
seed = 42
batch_size = 16
num_epochs = 5
learning_rate = 5e-5
weight_decay = 0.05
warmup_ratio = 0.1

# Create correct directory to the image file
image_root = "./classification_dataset/images/"
pairs = [(image_root + features[i], labels[i]) for i in range(len(features))]
print(pairs[:20])
image_dir = os.path.join(image_root, "images")
labels_path = os.path.join(image_root, "labels.txt")

# Classes
classes = sorted(list({label for _, label in pairs}))
label2id = {c: i for i, c in enumerate(classes)}
id2label = {i: c for c, i in label2id.items()}
# print(label2id)
# print(id2label)

# Optional: stratified split (recommended)
if HAVE_SKLEARN:
    X = [p for p, _ in pairs]
    y = [label2id[lbl] for _, lbl in pairs]
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        X, y, test_size=0.1, random_state=seed, stratify=y
    )
    train_pairs = list(zip(train_paths, [id2label[i] for i in train_labels]))
    val_pairs   = list(zip(val_paths,   [id2label[i] for i in val_labels]))
else:
    # fallback: simple shuffle + 90/10 split
    random.shuffle(pairs)
    k = max(1, int(0.1 * len(pairs)))
    val_pairs = pairs[:k]
    train_pairs = pairs[k:]

print(f"Train: {len(train_pairs)} | Val: {len(val_pairs)} | Classes: {classes}")

# ---- load model/processor once ----
CKPT = "./vit-single-out/best"
BASE = "google/vit-base-patch16-224"   # fallback for processor if needed

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForImageClassification.from_pretrained(CKPT).to(device).eval()

# ---- loop & score ----
correct = 0
total = len(val_pairs)

with torch.inference_mode():
    for fname, true_label in tqdm(val_pairs, total=total):
        img_path = fname
        img = Image.open(img_path).convert("RGB")
        inputs = processor(images=img, return_tensors="pt").to(device)  # <-- move to GPU/CPU
        logits = model(**inputs).logits
        pred_id = int(torch.argmax(logits, dim=-1))
        pred_label = model.config.id2label[pred_id]
        if pred_label == true_label:
            correct += 1

acc = 100.0 * correct / total
print(f"Accuracy: {acc:.2f}%  ({correct}/{total})")




[('./classification_dataset/images/2015_9.jpg', 'roundfish'), ('./classification_dataset/images/2015_18.jpg', 'whelk'), ('./classification_dataset/images/2015_32.jpg', 'roundfish'), ('./classification_dataset/images/2015_40.jpg', 'skate'), ('./classification_dataset/images/2015_45.jpg', 'roundfish'), ('./classification_dataset/images/2015_47.jpg', 'skate'), ('./classification_dataset/images/2015_102.jpg', 'Scallop'), ('./classification_dataset/images/2015_117.jpg', 'flatfish'), ('./classification_dataset/images/2015_126.jpg', 'roundfish'), ('./classification_dataset/images/2015_128.jpg', 'Scallop'), ('./classification_dataset/images/2015_136.jpg', 'roundfish'), ('./classification_dataset/images/2015_142.jpg', 'roundfish'), ('./classification_dataset/images/2015_151.jpg', 'flatfish'), ('./classification_dataset/images/2015_155.jpg', 'whelk'), ('./classification_dataset/images/2015_156.jpg', 'whelk'), ('./classification_dataset/images/2015_157.jpg', 'Scallop'), ('./classification_dataset

100%|██████████| 1050/1050 [02:46<00:00,  6.31it/s]

Accuracy: 92.29%  (969/1050)


In [ ]:
# Scoring accuracy over all images
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import torch
from tqdm import tqdm

# ---- read & parse labels ----
with open("./drive/MyDrive/classification_dataset/labels.txt", "r") as f:
    lines = [ln.strip() for ln in f if ln.strip()]

def sort_key(line):
    # assumes filenames like "2015_26395.jpg label"
    fn = line.split()[0]                # "2015_26395.jpg"
    prefix, rest = fn.split("_", 1)     # "2015", "26395.jpg"
    number = int(rest.split(".")[0])    # 26395
    return (int(prefix), number)

lines.sort(key=sort_key)
features = [ln.split()[0] for ln in lines]      # filenames
labels   = [ln.split()[1] for ln in lines]      # string labels

# ---- load model/processor once ----
CKPT = "./vit-single-out/best"
BASE = "google/vit-base-patch16-224"   # fallback for processor if needed

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# If your CKPT folder has no preprocessor_config.json, load processor from BASE
try:
    processor = AutoImageProcessor.from_pretrained(CKPT)
except Exception:
    processor = AutoImageProcessor.from_pretrained(BASE)

model = AutoModelForImageClassification.from_pretrained(CKPT).to(device).eval()

# ---- loop & score ----
correct = 0
total = len(features)

with torch.inference_mode():
    for fname, true_label in tqdm(zip(features, labels), total=total):
        img_path = f"./drive/MyDrive/classification_dataset/images/{fname}"
        img = Image.open(img_path).convert("RGB")
        inputs = processor(images=img, return_tensors="pt").to(device)  # <-- move to GPU/CPU
        logits = model(**inputs).logits
        pred_id = int(torch.argmax(logits, dim=-1))
        pred_label = model.config.id2label[pred_id]
        if pred_label == true_label:
            correct += 1

acc = 100.0 * correct / total
print(f"Accuracy: {acc:.2f}%  ({correct}/{total})")
